In [1]:
spark

In [ ]:
! pip3 install py4j==0.10.7
#! pip3 install numpy==1.13.3 --user
! pip3 install xgboost --user
! pip3 install pdpbox --user

In [2]:
import pandas as pd
import numpy as np
#from sklearn import tree
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.model_selection import train_test_split
#from pyspark.ml.classification import DecisionTreeClassifier

In [3]:
#Read data sets and view training data schema
training_data = spark.read.csv("train.csv", inferSchema=True, header=True)
testing_data = spark.read.csv("test.csv", inferSchema=True, header=True)
training_data.printSchema()

root
 |-- Id: string (nullable = true)
 |-- v2a1: double (nullable = true)
 |-- hacdor: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- hacapo: integer (nullable = true)
 |-- v14a: integer (nullable = true)
 |-- refrig: integer (nullable = true)
 |-- v18q: integer (nullable = true)
 |-- v18q1: integer (nullable = true)
 |-- r4h1: integer (nullable = true)
 |-- r4h2: integer (nullable = true)
 |-- r4h3: integer (nullable = true)
 |-- r4m1: integer (nullable = true)
 |-- r4m2: integer (nullable = true)
 |-- r4m3: integer (nullable = true)
 |-- r4t1: integer (nullable = true)
 |-- r4t2: integer (nullable = true)
 |-- r4t3: integer (nullable = true)
 |-- tamhog: integer (nullable = true)
 |-- tamviv: integer (nullable = true)
 |-- escolari: integer (nullable = true)
 |-- rez_esc: integer (nullable = true)
 |-- hhsize: integer (nullable = true)
 |-- paredblolad: integer (nullable = true)
 |-- paredzocalo: integer (nullable = true)
 |-- paredpreb: integer (nullable = tru

In [4]:
#Prepare data sets and view data
df = training_data.drop('Id')

In [5]:
df.toPandas().head()

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,190000.0,0,3,0,1,1,0,NaN,0,1,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,135000.0,0,4,0,1,1,1,1.0,0,1,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,NaN,0,8,0,1,1,0,NaN,0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,180000.0,0,5,0,1,1,1,1.0,0,2,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,180000.0,0,5,0,1,1,1,1.0,0,2,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [6]:
#Drop column with only one value for error handling in vectorization step
df = df.drop('elimbasu5')
df.toPandas().head()

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,190000.0,0,3,0,1,1,0,NaN,0,1,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,135000.0,0,4,0,1,1,1,1.0,0,1,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,NaN,0,8,0,1,1,0,NaN,0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,180000.0,0,5,0,1,1,1,1.0,0,2,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,180000.0,0,5,0,1,1,1,1.0,0,2,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [12]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

categoricalColumns = df.columns[0:140]
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
    
label_stringIdx = StringIndexer(inputCol = 'Target', outputCol = 'labels').setHandleInvalid('skip')
stages += [label_stringIdx]
numericCols = [item[0] for item in df.dtypes if item[1].startswith('integer')]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [8]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = df.columns
df = df.select(selectedCols)
df.printSchema()

root
 |-- v2a1: double (nullable = true)
 |-- hacdor: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- hacapo: integer (nullable = true)
 |-- v14a: integer (nullable = true)
 |-- refrig: integer (nullable = true)
 |-- v18q: integer (nullable = true)
 |-- v18q1: integer (nullable = true)
 |-- r4h1: integer (nullable = true)
 |-- r4h2: integer (nullable = true)
 |-- r4h3: integer (nullable = true)
 |-- r4m1: integer (nullable = true)
 |-- r4m2: integer (nullable = true)
 |-- r4m3: integer (nullable = true)
 |-- r4t1: integer (nullable = true)
 |-- r4t2: integer (nullable = true)
 |-- r4t3: integer (nullable = true)
 |-- tamhog: integer (nullable = true)
 |-- tamviv: integer (nullable = true)
 |-- escolari: integer (nullable = true)
 |-- rez_esc: integer (nullable = true)
 |-- hhsize: integer (nullable = true)
 |-- paredblolad: integer (nullable = true)
 |-- paredzocalo: integer (nullable = true)
 |-- paredpreb: integer (nullable = true)
 |-- pareddes: integer (nullabl

In [9]:
#Split data into 70% train, 30% validation (test) split, set seed for reproducibility
train, test = df.randomSplit([0.7, 0.3], seed = 2018)

In [10]:
#Fit data to Decision Tree Classifier
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'labels', maxDepth = 65)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select('tamhog', 'escolari', 'labels', 'rawPrediction', 'prediction', 'probability').show(10)

Py4JJavaError: An error occurred while calling o15677.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 288.0 failed 1 times, most recent failure: Lost task 0.0 in stage 288.0 (TID 289, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.writeFields_0_34$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iter

In [ ]:
#Evaluate Decision Tree model
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))